In [19]:
import fitbit
import gather_keys_oauth2 as Oauth2
import pandas as pd 
import datetime
import csv

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

# Create random data with numpy
import numpy as np
from collections import deque

In [20]:
CLIENT_ID = '22DPNX'
CLIENT_SECRET = '518e75ff5f011d25e392fcfdd26414dd'

In [22]:
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()
ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN)

[12/May/2019:13:10:00] ENGINE Listening for SIGTERM.
[12/May/2019:13:10:00] ENGINE Listening for SIGHUP.
[12/May/2019:13:10:00] ENGINE Listening for SIGUSR1.
[12/May/2019:13:10:00] ENGINE Bus STARTING
[12/May/2019:13:10:00] ENGINE Started monitor thread 'Autoreloader'.
[12/May/2019:13:10:00] ENGINE Serving on http://127.0.0.1:8080
[12/May/2019:13:10:00] ENGINE Bus STARTED


127.0.0.1 - - [12/May/2019:13:10:03] "GET /?code=cb1f8ff9d60a47c521821282d080d8fb2404453e&state=OEQGKmXDHzCtBGnqZkePo3j2iVyTdM HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36"


[12/May/2019:13:10:04] ENGINE Bus STOPPING
[12/May/2019:13:10:09] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[12/May/2019:13:10:09] ENGINE Stopped thread 'Autoreloader'.
[12/May/2019:13:10:09] ENGINE Bus STOPPED
[12/May/2019:13:10:09] ENGINE Bus EXITING
[12/May/2019:13:10:09] ENGINE Bus EXITED
[12/May/2019:13:10:09] ENGINE Waiting for child threads to terminate...


### Database layer

In [28]:
# https://github.com/yashatgit/fitbit-analyzer/blob/master/Fitbit_Data_Collection.ipynb

DATE_FORMAT = '%Y-%m-%d'
databaseFilename = 'Data/database_main.xls'

def connectAndLoadDb():
    print("Connecting database...")
    database = pd.read_excel(databaseFilename)
    print("Database connected!")
    return database;

def getLastEntryDate(database):
    lastDateEntry = database.iloc[-1]['Date']
    lastDateEntry = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)    
    return lastDateEntry

def addEntriesInDB(dictionary, database):
    #print(dictionary)
    database = database.append(dictionary, ignore_index=True)
    return addEntriesInDB;

def writeDbToExcelFile(database):
    print('Writing database to filename: '+ databaseFilename)
    writer = pd.ExcelWriter(databaseFilename)
    database.to_excel(writer, 'main')
    writer.save()
    print('Database updated with new entries!!')
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);

### Activity and Sleep Parsers

In [29]:
#x[4] if len(x) == 4 else 'No'
def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  

def percent(val):
    return np.ceil(val*100)

def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {} 

def mergeDicts(dicts):
    super_dict = {}
    for singleDict in dicts:
        for k, v in singleDict.items(): 
            super_dict[k] = v
    return super_dict

    
def getActivities(date):
    activitiyResponse = auth2_client.activities(date=date)
    
    activitySummary = activitiyResponse['summary'];
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Elevation (Ft)':activitySummary['elevation'],
        'Floors':activitySummary['floors'],
        'Minutes Sedentary':activitySummary['sedentaryMinutes'],
        'Minutes Lightly Active':activitySummary['lightlyActiveMinutes'],
        'Minutes Fairly Active':activitySummary['fairlyActiveMinutes'],
        'Minutes Very Active':activitySummary['veryActiveMinutes'],
        'Activity Calories': activitySummary['activityCalories'],
        'Active Score': activitySummary['activeScore'],
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    return activityData

def getSleep(date):
    sleepResponse = auth2_client_new.sleep(date=date)
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepLog.get('levels', {}).get('summary', {})
            
            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')
            
            sleepData['Minutes Deep sleep'] = safeGet(sleepLevelsSummary.get('deep', {}), 'minutes')
            sleepData['Deep sleep count'] = safeGet(sleepLevelsSummary.get('deep', {}), 'count')
            sleepData['% Deep sleep'] = percent(safeGet(sleepData, 'Minutes Deep sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Light sleep'] = safeGet(sleepLevelsSummary.get('light', {}), 'minutes')
            sleepData['Light sleep count'] = safeGet(sleepLevelsSummary.get('light', {}), 'count')
            sleepData['% Light sleep'] = percent(safeGet(sleepData, 'Minutes Light sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes REM sleep'] = safeGet(sleepLevelsSummary.get('rem', {}), 'minutes')
            sleepData['REM sleep count'] = safeGet(sleepLevelsSummary.get('rem', {}), 'count')
            sleepData['% REM sleep'] = percent(safeGet(sleepData, 'Minutes REM sleep', 0)/safeGet(sleepData, 'Time in bed', 0))
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = safeGet(sleepLevelsSummary.get('wake', {}), 'minutes')
            sleepData['Minutes Awake count'] = safeGet(sleepLevelsSummary.get('wake', {}), 'count')            
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5,
        'Date': prettyPrintDate(date)
    };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date + timedelta(days=1)
    sleepData = getSleep(prettyPrintDate(nextDate))
    
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    mergedData = mergeDicts([sleepData, activitiesData, dateData])
    return mergedData


### Data fetch layer

In [30]:
database = connectAndLoadDb()

def shouldFetchDataForProvidedDate(providedDate, todaysDate, API_COUNTER):
    return (providedDate < todaysDate) and API_COUNTER < 100;

def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    database = database.append(mergedData, ignore_index=True)
    return database;

def fetchData(database, refetchAll = False):
    API_COUNTER = 0
    
    todaysDate = datetime.datetime.today()
    
    print("Date today is :" + prettyPrintDate(todaysDate))
    
    if refetchAll == True:
        lastEntryDate = datetime.datetime.strptime('2019-05-12', DATE_FORMAT)
    else:
        lastEntryDate = getLastEntryDate(database)
    
    print("Last entry in Db is of :" + prettyPrintDate(lastEntryDate))
    print("----------------------------------------------")
    
    dateToFetch = lastEntryDate + timedelta(days=1)

    while shouldFetchDataForProvidedDate(dateToFetch, todaysDate, API_COUNTER):    
        database = fetchAndAppendToDb(dateToFetch, database)
        dateToFetch = dateToFetch + timedelta(days=1)
        API_COUNTER = API_COUNTER+1
    
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return database

Connecting database...


XLRDError: File size is 0 bytes

In [13]:
yesterday = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
yesterday2 = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
today = str(datetime.datetime.now().strftime("%Y%m%d"))

In [14]:
fit_statsHR = auth2_client.intraday_time_series('activities/heart', base_date=yesterday2, detail_level='1sec')

In [8]:
fit_statsHR['activities-heart-intraday']

{'dataset': [{'time': '00:00:04', 'value': 60},
  {'time': '00:00:14', 'value': 59},
  {'time': '00:00:19', 'value': 60},
  {'time': '00:00:34', 'value': 60},
  {'time': '00:00:39', 'value': 61},
  {'time': '00:00:44', 'value': 62},
  {'time': '00:00:54', 'value': 60},
  {'time': '00:00:59', 'value': 57},
  {'time': '00:01:04', 'value': 55},
  {'time': '00:01:09', 'value': 54},
  {'time': '00:01:14', 'value': 58},
  {'time': '00:01:19', 'value': 61},
  {'time': '00:01:34', 'value': 61},
  {'time': '00:01:44', 'value': 60},
  {'time': '00:01:49', 'value': 61},
  {'time': '00:02:04', 'value': 61},
  {'time': '00:02:10', 'value': 63},
  {'time': '00:02:15', 'value': 62},
  {'time': '00:02:25', 'value': 63},
  {'time': '00:02:30', 'value': 64},
  {'time': '00:02:35', 'value': 63},
  {'time': '00:02:40', 'value': 62},
  {'time': '00:02:45', 'value': 61},
  {'time': '00:02:50', 'value': 57},
  {'time': '00:02:55', 'value': 58},
  {'time': '00:03:00', 'value': 62},
  {'time': '00:03:05', 'val

In [8]:
time_list = []
val_list = []
for i in fit_statsHR['activities-heart-intraday']['dataset']:
    val_list.append(i['value'])
    time_list.append(i['time'])
heartdf = pd.DataFrame({'Heart Rate':val_list,'Time':time_list})

In [9]:
heartdf.sample(1)

,Heart Rate,Time
349,72,07:48:42


In [10]:
heartdf.to_csv('/Users/helena/Downloads/fitbit-heart'+ \
               yesterday+'.csv', \
               columns=['Time','Heart Rate'], header=True, \
               index = False)

In [9]:
# My yesterday heartrate
heartrate = heartdf['Heart Rate']
time = heartdf['Time']
trace = go.Scatter(
    x = time,
    y = heartrate,
    mode = 'lines',
    name = 'heart beat',
        line = dict(
        color = ('rgb(178,34,34)'),
        width = 1)
)
data=[trace]
layout = go.Layout(
    title='one day heartbeat data',
    xaxis=dict(
        title='time',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=1,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=1,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        title='heartbeat',
        showgrid=False,
        zeroline=False,
        showline=False,
#         showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

NameError: name 'heartdf' is not defined

In [13]:
fit_statsAC = auth2_client.intraday_time_series('activities/calories', base_date=yesterday2, detail_level='1min')

#     cals = authd_client.time_series('activities/calories', base_date= last_log_date_str ,end_date=yesterday_str)
#     dist = authd_client.time_series('activities/distance', base_date= last_log_date_str ,end_date=yesterday_str)
#     floors = authd_client.time_series('activities/floors', base_date= last_log_date_str ,end_date=yesterday_str)
#     sedant = authd_client.time_series('activities/minutesSedentary', base_date= last_log_date_str ,end_date=yesterday_str)
#     elevation = authd_client.time_series('activities/elevation', base_date= last_log_date_str ,end_date=yesterday_str)
#     active_light = authd_client.time_series('activities/minutesLightlyActive', base_date= last_log_date_str ,end_date=yesterday_str)
#     active_fair = authd_client.time_series('activities/minutesFairlyActive', base_date= last_log_date_str ,end_date=yesterday_str)
#     active_very = authd_client.time_series('activities/minutesVeryActive', base_date= last_log_date_str ,end_date=yesterday_str)
#     active_cals = authd_client.time_series('activities/activityCalories', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_start = authd_client.time_series('sleep/startTime', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_timeInBed = authd_client.time_series('sleep/timeInBed', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_minutesAsleep = authd_client.time_series('sleep/minutesAsleep', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_awakeningsCount = authd_client.time_series('sleep/awakeningsCount', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_minutesAwake = authd_client.time_series('sleep/minutesAwake', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_minutesToFallAsleep = authd_client.time_series('sleep/minutesToFallAsleep', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_minutesAfterWakeup = authd_client.time_series('sleep/minutesAfterWakeup', base_date= last_log_date_str ,end_date=yesterday_str)
#     sleep_efficiency = authd_client.time_series('sleep/efficiency', base_date= last_log_date_str ,end_date=yesterday_str)


In [15]:
steps = auth2_client.intraday_time_series('activities/steps', base_date=yesterday2, detail_level='1min')

In [16]:
steps


{'activities-steps': [{'dateTime': '2019-05-08', 'value': '19144'}],
 'activities-steps-intraday': {'dataset': [{'time': '00:00:00', 'value': 0},
   {'time': '00:01:00', 'value': 0},
   {'time': '00:02:00', 'value': 0},
   {'time': '00:03:00', 'value': 0},
   {'time': '00:04:00', 'value': 0},
   {'time': '00:05:00', 'value': 0},
   {'time': '00:06:00', 'value': 0},
   {'time': '00:07:00', 'value': 0},
   {'time': '00:08:00', 'value': 0},
   {'time': '00:09:00', 'value': 0},
   {'time': '00:10:00', 'value': 0},
   {'time': '00:11:00', 'value': 0},
   {'time': '00:12:00', 'value': 0},
   {'time': '00:13:00', 'value': 0},
   {'time': '00:14:00', 'value': 0},
   {'time': '00:15:00', 'value': 0},
   {'time': '00:16:00', 'value': 0},
   {'time': '00:17:00', 'value': 0},
   {'time': '00:18:00', 'value': 0},
   {'time': '00:19:00', 'value': 0},
   {'time': '00:20:00', 'value': 0},
   {'time': '00:21:00', 'value': 0},
   {'time': '00:22:00', 'value': 0},
   {'time': '00:23:00', 'value': 0},
   

In [22]:
time_list = []
val_list = []
mets_list = []
for i in fit_statsAC['activities-calories-intraday']['dataset']:
    val_list.append(i['value'])
    time_list.append(i['time'])
    mets_list.append(i['mets'])
caloriesdf = pd.DataFrame({'Calories':val_list,'Time':time_list, 'mets': mets_list})

In [ ]:
time_list = []
val_list = []
for i in fit_statsHR['activities-heart-intraday']['dataset']:
    val_list.append(i['value'])
    time_list.append(i['time'])
heartdf = pd.DataFrame({'Heart Rate':val_list,'Time':time_list})

In [23]:
caloriesdf.sample(1)

,Calories,Time,mets
369,1.0142,06:09:00,10


In [27]:
# My yesterday calories
calories = caloriesdf['Calories']
time = caloriesdf['Time']

trace = go.Scatter(
    x = time,
    y = calories,
    mode = 'lines',
    name = 'calories',
        line = dict(
        color = ('rgb(44,127,184)'),
        width = 1)
)
data=[trace]
layout = go.Layout(
    title='one day calories data',
    xaxis=dict(
        title='minute',
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        title='calories',
        showgrid=False,
        zeroline=False,
        showline=False,
#         showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False
)


fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [18]:
steps = auth2_client.intraday_time_series('activities/steps', base_date=yesterday2, detail_level='1min')
cals = auth2_client.intraday_time_series('activities/calories', base_date=yesterday2, detail_level='1min')
dist = auth2_client.intraday_time_series('activities/distance', base_date=yesterday2, detail_level='1min')
floors  = auth2_client.intraday_time_series('activities/floors', base_date=yesterday2, detail_level='1min')
sedant = auth2_client.intraday_time_series('activities/minutesSedentary', base_date=yesterday2, detail_level='1min')
elevation = auth2_client.intraday_time_series('activities/elevation', base_date=yesterday2, detail_level='1min')
active_light = auth2_client.intraday_time_series('activities/minutesLightlyActive', base_date=yesterday2, detail_level='1min')
active_fair = auth2_client.intraday_time_series('activities/minutesFairlyActive', base_date=yesterday2, detail_level='1min')
active_very = auth2_client.intraday_time_series('activities/minutesVeryActive', base_date=yesterday2, detail_level='1min')



In [ ]:
yesterday = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
yesterday2 = str((datetime.datetime.now() - datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
today = str(datetime.datetime.now().strftime("%Y%m%d"))

In [23]:
df = pd.DataFrame()
num_days = yesterday - today + timedelta(1)
date_list = [last_log_date.date() + timedelta(days=x) for x in range(0, num_days.days)]
df['date'] = date_list
df['calories'] = pd.DataFrame(cals['activities-calories'])['value'].astype(int)
df['steps'] = pd.DataFrame(steps['activities-steps'])['value'].astype(int)
df['dist'] = pd.DataFrame(dist['activities-distance'])['value'].astype(float)
df['floors'] = pd.DataFrame(floors['activities-floors'])['value'].astype(int)
df['elevation'] = pd.DataFrame(elevation['activities-elevation'])['value'].astype(float)
df['sedant'] = pd.DataFrame(sedant['activities-minutesSedentary'])['value'].astype(int)
df['active_light'] = pd.DataFrame(active_light['activities-minutesLightlyActive'])['value'].astype(int)
df['active_fair'] = pd.DataFrame(active_fair['activities-minutesFairlyActive'])['value'].astype(int)
df['active_very'] = pd.DataFrame(active_very['activities-minutesVeryActive'])['value'].astype(int)
#     df['active_cals'] = pd.DataFrame(active_cals['activities-activityCalories'])['value'].astype(int)
#     df['sleep_start'] = pd.DataFrame(sleep_start['sleep-startTime'])['value']
#     df['sleep_timeInBed'] = pd.DataFrame(sleep_timeInBed['sleep-timeInBed'])['value'].astype(int)
#     df['sleep_minutesAsleep'] = pd.DataFrame(sleep_minutesAsleep['sleep-minutesAsleep'])['value'].astype(int)
#     df['sleep_awakeningsCount'] = pd.DataFrame(sleep_awakeningsCount['sleep-awakeningsCount'])['value'].astype(int)
#     df['sleep_minutesAwake'] = pd.DataFrame(sleep_minutesAwake['sleep-minutesAwake'])['value'].astype(int)
#     df['sleep_minutesToFallAsleep'] = pd.DataFrame(sleep_minutesToFallAsleep['sleep-minutesToFallAsleep'])['value'].astype(int)
#     df['sleep_minutesAfterWakeup'] = pd.DataFrame(sleep_minutesAfterWakeup['sleep-minutesAfterWakeup'])['value'].astype(int)
#     df['sleep_efficiency'] = pd.DataFrame(sleep_efficiency['sleep-efficiency'])['value'].astype(int)
return df

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [51]:
# Function to get the latest Fitbit data from the API.

def get_latest_fitbit_data(authd_client, last_log_date, yesterday):
    
    last_log_date_str = datetime.strftime(last_log_date,"%Y-%m-%d")
    yesterday_str = datetime.strftime(yesterday,"%Y-%m-%d")
    
    # Retrieve time series data by accessing Fitbit API. 
    # Note that the last logged date is the first date because data only upto yesterday gets written into csv.
    # Today is not over, hence incomplete data from today is not logged.

    steps = authd_client.time_series('activities/steps', base_date= last_log_date_str ,end_date=yesterday_str)
    cals = authd_client.time_series('activities/calories', base_date= last_log_date_str ,end_date=yesterday_str)
    dist = authd_client.time_series('activities/distance', base_date= last_log_date_str ,end_date=yesterday_str)
    floors = authd_client.time_series('activities/floors', base_date= last_log_date_str ,end_date=yesterday_str)
    sedant = authd_client.time_series('activities/minutesSedentary', base_date= last_log_date_str ,end_date=yesterday_str)
    elevation = authd_client.time_series('activities/elevation', base_date= last_log_date_str ,end_date=yesterday_str)
    active_light = authd_client.time_series('activities/minutesLightlyActive', base_date= last_log_date_str ,end_date=yesterday_str)
    active_fair = authd_client.time_series('activities/minutesFairlyActive', base_date= last_log_date_str ,end_date=yesterday_str)
    active_very = authd_client.time_series('activities/minutesVeryActive', base_date= last_log_date_str ,end_date=yesterday_str)
    active_cals = authd_client.time_series('activities/activityCalories', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_start = authd_client.time_series('sleep/startTime', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_timeInBed = authd_client.time_series('sleep/timeInBed', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_minutesAsleep = authd_client.time_series('sleep/minutesAsleep', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_awakeningsCount = authd_client.time_series('sleep/awakeningsCount', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_minutesAwake = authd_client.time_series('sleep/minutesAwake', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_minutesToFallAsleep = authd_client.time_series('sleep/minutesToFallAsleep', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_minutesAfterWakeup = authd_client.time_series('sleep/minutesAfterWakeup', base_date= last_log_date_str ,end_date=yesterday_str)
    sleep_efficiency = authd_client.time_series('sleep/efficiency', base_date= last_log_date_str ,end_date=yesterday_str)

    df = pd.DataFrame()
    num_days = yesterday - last_log_date + timedelta(1)
    date_list = [last_log_date.date() + timedelta(days=x) for x in range(0, num_days.days)]
    df['date'] = date_list
    df['calories'] = pd.DataFrame(cals['activities-calories'])['value'].astype(int)
    df['steps'] = pd.DataFrame(steps['activities-steps'])['value'].astype(int)
    df['dist'] = pd.DataFrame(dist['activities-distance'])['value'].astype(float)
    df['floors'] = pd.DataFrame(floors['activities-floors'])['value'].astype(int)
    df['elevation'] = pd.DataFrame(elevation['activities-elevation'])['value'].astype(float)
    df['sedant'] = pd.DataFrame(sedant['activities-minutesSedentary'])['value'].astype(int)
    df['active_light'] = pd.DataFrame(active_light['activities-minutesLightlyActive'])['value'].astype(int)
    df['active_fair'] = pd.DataFrame(active_fair['activities-minutesFairlyActive'])['value'].astype(int)
    df['active_very'] = pd.DataFrame(active_very['activities-minutesVeryActive'])['value'].astype(int)
    df['active_cals'] = pd.DataFrame(active_cals['activities-activityCalories'])['value'].astype(int)
    df['sleep_start'] = pd.DataFrame(sleep_start['sleep-startTime'])['value']
    df['sleep_timeInBed'] = pd.DataFrame(sleep_timeInBed['sleep-timeInBed'])['value'].astype(int)
    df['sleep_minutesAsleep'] = pd.DataFrame(sleep_minutesAsleep['sleep-minutesAsleep'])['value'].astype(int)
    df['sleep_awakeningsCount'] = pd.DataFrame(sleep_awakeningsCount['sleep-awakeningsCount'])['value'].astype(int)
    df['sleep_minutesAwake'] = pd.DataFrame(sleep_minutesAwake['sleep-minutesAwake'])['value'].astype(int)
    df['sleep_minutesToFallAsleep'] = pd.DataFrame(sleep_minutesToFallAsleep['sleep-minutesToFallAsleep'])['value'].astype(int)
    df['sleep_minutesAfterWakeup'] = pd.DataFrame(sleep_minutesAfterWakeup['sleep-minutesAfterWakeup'])['value'].astype(int)
    df['sleep_efficiency'] = pd.DataFrame(sleep_efficiency['sleep-efficiency'])['value'].astype(int)
    return df

In [52]:

def write_fitbit_data_to_csv(df,data_filename,log_filename):
    # Write the data frame to csv.
    with open(data_filename, 'a') as f:
        df.to_csv(f, header=False,index = False)

    # Log current date as the date last logged.
    today = datetime.today()
    today_str = datetime.strftime(today,"%Y-%m-%d")

    with open(log_filename,'ab') as csvfile:
        csvwrite = csv.writer(csvfile, delimiter=',')
        csvwrite.writerow(['Data last logged',today_str])
        
        
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n) 

def get_intraday_data(authd_client,authd_client2,start_date,end_date):
    rate_limit = 150
    df_master = pd.DataFrame()
    
    num_days = end_date - start_date
    
    # If the number of calls is going to exceed the rate limit, then change end date to last feasible date
    if num_days.days > int((rate_limit-18)/7):
        end_date = start_date + timedelta(int((rate_limit-18)/7))
        print('Date range causes rate limit to exceed')
    else:
        try:
            for single_date in daterange(start_date, end_date+timedelta(1)):
                # Create a separate dataframe for intra day data
                df_intra = pd.DataFrame(columns=['time_intra','steps_intra','calories_intra','distance_intra'\
                                                 'elevation_intra','floors_intra','date'])
                df_hr_intra = pd.DataFrame(columns=['time_intra','hr_intra'])
                df_sleep_intra = pd.DataFrame(columns=['time_intra','sleep_intra'])

                # Get intra day data using oauth client
                steps_intra = authd_client.intraday_time_series('activities/steps',base_date=single_date)
                calories_intra = authd_client.intraday_time_series('activities/calories',base_date=single_date)
                distance_intra = authd_client.intraday_time_series('activities/distance',base_date=single_date)
                elevation_intra = authd_client.intraday_time_series('activities/elevation',base_date=single_date)
                floors_intra = authd_client.intraday_time_series('activities/floors',base_date=single_date)

                # Intra sleep data
                sleep_intra = authd_client.get_sleep(single_date)

                # Heart rate using oauth2.0
                try:
                    hr_intra = authd_client2.intraday_time_series('activities/heart',base_date=single_date)
                except fitbit.exceptions.HTTPUnauthorized:
                    print('Please provide latest refresh and access tokens for oauth2. Exiting program.')
                    sys.exit(1)
                 
                # Temperory pandas dataframes hold all the data before being merged to one big dataframe
                df_intra['time_intra'] = pd.DataFrame(steps_intra['activities-steps-intraday']['dataset'])['time']
                df_intra['steps_intra'] = pd.DataFrame(steps_intra['activities-steps-intraday']['dataset'])['value'].astype(int)
                df_intra['calories_intra'] = pd.DataFrame(calories_intra['activities-calories-intraday']['dataset'])['value'].astype(int)
                df_intra['distance_intra'] = pd.DataFrame(distance_intra['activities-distance-intraday']['dataset'])['value'].astype(float)
                df_intra['elevation_intra'] = pd.DataFrame(elevation_intra['activities-elevation-intraday']['dataset'])['value'].astype(float)
                df_intra['floors_intra'] = pd.DataFrame(floors_intra['activities-floors-intraday']['dataset'])['value'].astype(int)
                df_intra['date'] = single_date.date()
                df_hr_intra['time_intra'] = pd.DataFrame(hr_intra['activities-heart-intraday']['dataset'])['time']
                df_hr_intra['hr_intra'] = pd.DataFrame(hr_intra['activities-heart-intraday']['dataset'])['value'].astype(int)

                # Some days don't have sleep information if Fitbit was not worn before going to bed (or for other reasons)
                if not sleep_intra['sleep']:
                    print (['No sleep data for ',str(datetime.strftime(single_date,"%Y-%m-%d"))])
                else:
                    df_sleep_intra['time_intra'] = pd.DataFrame(sleep_intra['sleep'][0]['minuteData'])['dateTime']
                    df_sleep_intra['sleep_intra'] = pd.DataFrame(sleep_intra['sleep'][0]['minuteData'])['value'].astype(int)

                # Merge the two data frames into one. Note the reason for doing it this way is because the heart rate data does not necessarily 
                # have all the minutes. Whenever the Fitbit has been removed and kept aside, no HR data was collected 
                df_intra = pd.merge(df_intra,df_hr_intra, how='left', on='time_intra',
                      left_index=False, right_index=False, sort=False,
                      suffixes=('_x', '_y'), copy=True)

                df_intra = pd.merge(df_intra,df_sleep_intra, how='left', on='time_intra',
                      left_index=False, right_index=False, sort=False,
                      suffixes=('_x', '_y'), copy=True)

                # Concatenate to master dataframe
                df_master = pd.concat([df_master,df_intra], ignore_index = True)
        except fitbit.exceptions.HTTPTooManyRequests:
            print('Rate limit reached. Rerun program after 1 hour. Exiting program.')
            sys.exit(1)
            
    return df_master


def obtain_write_new_data():
    # Get Fitbit User keys and secrets from csv file
    auth_keys = dict()
    with open('Data/access_tokens.csv','r') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        for row in csv_reader:
            auth_keys[row[0]] = row[1]

    # Create an oauth and oauth2 client 
#     authd_client = fitbit.Fitbit(auth_keys['consumer_key'], auth_keys['consumer_secret'], resource_owner_key =auth_keys['user_key'], resource_owner_secret =auth_keys['user_secret'])
#     authd_client2 = fitbit.Fitbit(auth_keys['client_id'], auth_keys['client_secret'], oauth2=True, access_token=auth_keys['access_token'], refresh_token=auth_keys['refresh_token'])

    # Read log file to find out the last day Fitbit data was logged
    with open('Data/Data_log_dates.csv') as csvfile:
        temp = deque(csv.reader(csvfile), 1)[0]

    # Get yesterday's date and last logged date in both datetime format and in string format
    # Data from today is incomplete and not logged. So we are interested in yesterday's date.

    last_log_date = datetime.strptime(temp[1],"%Y-%m-%d")
    today = datetime.today()
    yesterday = today - timedelta(days=1) 

    # Get latest fitbit daily data
    df = get_latest_fitbit_data(authd_client,last_log_date,yesterday)

    # Add latest daily data to daily data csv file
    write_fitbit_data_to_csv(df,'Data/Daily_Fitbit_Data.csv','Data/Data_log_dates.csv')

    # Get latest intraday data
    # Read log file to find out the last day Fitbit data was logged
    with open('Data/Data_log_dates_intraday.csv') as csvfile:
        temp = deque(csv.reader(csvfile), 1)[0]
    last_log_date = datetime.strptime(temp[1],"%Y-%m-%d")

    # Get intraday data for all days from last_log_date to yesterday
    df_master = get_intraday_data(authd_client,authd_client2,last_log_date,yesterday)

    # Add latest daily intra day data to daily data csv file
    write_fitbit_data_to_csv(df_master,'Data/Intraday_data.csv','Data/Data_log_dates_intraday.csv')

In [54]:

# Call function to obtain and write new data
obtain_write_new_data()

IndexError: deque index out of range

In [35]:
df.sample(1)

NameError: name 'df' is not defined

In [ ]:
https://www.manishkurse.com/PythonProjects/Analyze_Fitbit_Data.html